# Université Paul Sabatier
**Etudiant : BERQUEZ Léa** 

EMIND1G1 - Fondements de la recherche d'information

**TP 3**

Enseignant : José G. Moreno

2020

## TP 3. Application de la RI pour la liaison référentielle 

Une application des méthodes de RI est la liaison référentielle. Cette tâche consiste à trouver le lien entre une portion de texte et la page Wikipédia qui le représentent le mieux. Dans ce cas d’application :
* Comme requêtes nous allons utiliser chaque mot dans une phrase (ou chaque mot important).
* Le corpus sont les pages Wikipédia. Donc, chaque page Wikipédia est un document à retrouver.
* La liste de résultats sont les candidats pour la liaison référentielle.

Considère par exemple la chaîne de caractères « ***Thomas and Mario are strikers playing in Munich*** ». Il existe trois sous-chaînes (***Thomas***, ***Mario*** et ***Munich***) ou mots importants qui sont susceptibles d'être identifiées dans une base de connaissances telle que Wikipédia. Une recherche de ces mots-clés dans les pages Wikipédia produirait de nombreux concepts candidats pour ces sous-chaînes. Donc un bon système doit retrouver comme premier résultats les liens aux pages Wikipédia : [Thomas Müller](https://en.wikipedia.org/wiki/Thomas_M%C3%BCller), [Mario Gómez](https://en.wikipedia.org/wiki/Mario_G%C3%B3mez) (ou Mario Götze) et FC [Bayern Munich](https://en.wikipedia.org/wiki/FC_Bayern_Munich). Cette tâche est particulièrement complexe et les méthodes de recherche d’information sont particulièrement utiles pour la résoudre.

> Plusieurs méthodes de liaison référentielle (Entity Linking en anglais) sont disponibles sur internet. Les plus connus sont : 

>	* Spotlight → https://dbpedia-spotlight.github.io/demo/
>	* Babelfy → http://babelfy.org/
>	* TagMe → https://tagme.d4science.org/tagme/

### 0) Mise en place de l'index 


In [1]:
# déclaration de la variable JAVA_HOME
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
!export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64

In [2]:
#installation de pyterrier avec pip
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

  Cloning https://github.com/terrier-org/pyterrier.git to /tmp/pip-install-n0payhei/python-terrier
  Running command git clone -q https://github.com/terrier-org/pyterrier.git /tmp/pip-install-n0payhei/python-terrier
     |████████████████████████████████| 1.1MB 6.1MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.3.0.dev0-cp36-none-any.whl size=54092 sha256=f4fce7e89874ffb3e9186c381a3391548ed3d6780d7d56869d645019726dd267
  Stored in directory: /tmp/pip-ephem-wheel-cache-37sdbx0_/wheels/cc/bb/69/836d846a92c787b35ca6478119c0033762ab2b95d866eeb288
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=69b0556c396c3e617528e2c469fc1e4f6db9a917e1d516754496a66fc99ed70a
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
  Created wheel for pytrec-eval: filename=pytrec_eval-0.5-cp36-cp36m-linux_x86_64.whl size=265284 sha256=b82cdba3de81bf6b19147

In [3]:
#Initialization de JVM
import pyterrier as pt
if not pt.started():
  pt.init()

terrier-assemblies 5.3  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.4  jar not found, downloading to /root/.pyterrier...
Done


**Installation de l'index** 

In [5]:
!wget https://dumps.wikimedia.org/simplewiki/20200901/simplewiki-20200901-abstract.xml.gz 

--2020-11-01 09:30:22--  https://dumps.wikimedia.org/simplewiki/20200901/simplewiki-20200901-abstract.xml.gz
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15445414 (15M) [application/octet-stream]
Saving to: ‘simplewiki-20200901-abstract.xml.gz’

simplewiki-20200901 100%[===================>]  14.73M  5.01MB/s    in 2.9s    

2020-11-01 09:30:26 (5.01 MB/s) - ‘simplewiki-20200901-abstract.xml.gz’ saved [15445414/15445414]



In [6]:
!gzip -d simplewiki-20200901-abstract.xml.gz #Décompresse le fichier  

In [7]:
import pandas as pd
import xml.etree.ElementTree as et

def parse_XML(xml_file, df_cols): 
    """Parse the input XML file and store the result in a pandas 
    DataFrame with the given columns. 
    
    The first element of df_cols is supposed to be the identifier 
    variable, which is an attribute of each node element in the 
    XML data; other features will be parsed from the text content 
    of each sub-element. 
    """
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    for node in xroot: 
        res = []
        for el in df_cols[0:]: 
            if node is not None and node.find(el) is not None:
                res.append(node.find(el).text)
            else: 
                res.append(None)
        rows.append({df_cols[i]: res[i] 
                     for i, _ in enumerate(df_cols)})
    
    out_df = pd.DataFrame(rows, columns=df_cols)
        
    return out_df 

In [8]:
!head -50 simplewiki-20200901-abstract.xml 

<feed>
<doc>
<title>Wikipedia: April</title>
<url>https://simple.wikipedia.org/wiki/April</url>
<abstract>April is the fourth month of the year, and comes between March and May. It is one of four months to have 30 days.</abstract>
<links>
<sublink linktype="nav"><anchor>The Month</anchor><link>https://simple.wikipedia.org/wiki/April#The_Month</link></sublink>
<sublink linktype="nav"><anchor>April in poetry</anchor><link>https://simple.wikipedia.org/wiki/April#April_in_poetry</link></sublink>
<sublink linktype="nav"><anchor>Events in April</anchor><link>https://simple.wikipedia.org/wiki/April#Events_in_April</link></sublink>
<sublink linktype="nav"><anchor>Fixed Events</anchor><link>https://simple.wikipedia.org/wiki/April#Fixed_Events</link></sublink>
<sublink linktype="nav"><anchor>Moveable Events</anchor><link>https://simple.wikipedia.org/wiki/April#Moveable_Events</link></sublink>
<sublink linktype="nav"><anchor>Selection of Historical Events</anchor><link>https://simple.wikipedia.or

In [9]:
df = parse_XML("simplewiki-20200901-abstract.xml", ["title", "url", "abstract"])
df["docno"]=[str(i+1) for i in range(len(df))]
df

,title,url,abstract,docno
0,Wikipedia: April,https://simple.wikipedia.org/wiki/April,"April is the fourth month of the year, and com...",1
1,Wikipedia: August,https://simple.wikipedia.org/wiki/August,August (Aug.) is the eighth month of the year ...,2
2,Wikipedia: Art,https://simple.wikipedia.org/wiki/Art,Category:Non-verbal communication,3
3,Wikipedia: A,https://simple.wikipedia.org/wiki/A,A or a is the first letter of the English alph...,4
4,Wikipedia: Air,https://simple.wikipedia.org/wiki/Air,thumb|A [is used to move air.],5
...,...,...,...,...
171492,Wikipedia: American comic book,https://simple.wikipedia.org/wiki/American_com...,An American comic book is a thin book in the U...,171493
171493,"Wikipedia: Calistoga, California","https://simple.wikipedia.org/wiki/Calistoga,_C...",|subdivision_type = Country,171494
171494,Wikipedia: Global citizenship,https://simple.wikipedia.org/wiki/Global_citiz...,Global citizenship is the idea that everyone i...,171495
171495,Wikipedia: Sidney Jellicoe,https://simple.wikipedia.org/wiki/Sidney_Jellicoe,Sidney Jellicoe (25 August 1906 – 24 November ...,171496


In [23]:
import pandas as pd
!rm -rf ./pd_indexTitre
!rm -rf ./pd_indexResume
pd_indexerTitre = pt.DFIndexer("./pd_indexTitre") 
pd_indexerResume = pt.DFIndexer("./pd_indexResume") 

In [24]:
indexrefTitre = pd_indexerTitre.index(df["title"], df["docno"], df["url"], df["title"]) 
indexrefResume = pd_indexerResume.index(df["abstract"], df["docno"], df["url"], df["title"]) 

09:39:41.093 [main] WARN  o.t.structures.indexing.Indexer - Adding an empty document to the index (29) - further warnings are suppressed
09:41:16.508 [main] WARN  o.t.structures.indexing.Indexer - Indexed 2883 empty documents


### 1) Liaison avec le texte
En utilisant l’indexe du champ <text> de Wikipédia et la requête « Thomas and Mario are strikers playing in Munich » vérifiez si vous obtenez les pages Wikipédia respectives :
* Thomas Müller
* Mario Gómez ou Mario Götze
* FC Bayern Munich


In [11]:
pt.BatchRetrieve(indexrefResume, wmodel="BM25", metadata=["docno","title","url"]).transform("Thomas and Mario are strikers playing in Munich").head(50)

,qid,docid,docno,title,url,rank,score,query
0,1,54535,54536,Wikipedia: Paper Mario,https://simple.wikipedia.org/wiki/Paper_Mario,0,18.699940,Thomas and Mario are strikers playing in Munich
1,1,98207,98208,Wikipedia: Christian Bentez,https://simple.wikipedia.org/wiki/Christian_Be...,1,17.387904,Thomas and Mario are strikers playing in Munich
2,1,71326,71327,Wikipedia: Gernot Jurtin,https://simple.wikipedia.org/wiki/Gernot_Jurtin,2,17.387904,Thomas and Mario are strikers playing in Munich
3,1,131294,131295,Wikipedia: Munich RE,https://simple.wikipedia.org/wiki/Munich_RE,3,16.909863,Thomas and Mario are strikers playing in Munich
4,1,104224,104225,Wikipedia: Enner Valencia,https://simple.wikipedia.org/wiki/Enner_Valencia,4,16.829015,Thomas and Mario are strikers playing in Munich
5,1,52660,52661,Wikipedia: Gary Twigg,https://simple.wikipedia.org/wiki/Gary_Twigg,5,16.829015,Thomas and Mario are strikers playing in Munich
6,1,20273,20274,Wikipedia: Mario (disambiguation),https://simple.wikipedia.org/wiki/Mario_(disam...,6,16.779104,Thomas and Mario are strikers playing in Munich
7,1,44657,44658,Wikipedia: Flrin Albert,https://simple.wikipedia.org/wiki/Fl%C3%B3ri%C...,7,16.405210,Thomas and Mario are strikers playing in Munich
8,1,144124,144125,Wikipedia: Jean-Claude Schindelholz,https://simple.wikipedia.org/wiki/Jean-Claude_...,8,16.198575,Thomas and Mario are strikers playing in Munich
9,1,122975,122976,"Wikipedia: Bobby Murdoch (footballer, born 1936)",https://simple.wikipedia.org/wiki/Bobby_Murdoc...,9,16.198575,Thomas and Mario are strikers playing in Munich


On n'obtient pas les pages attendus dans cette précédente requête. 

### 2) Les poids

Utilisez les différents méthodes de pondération prédefinies sur pyterrier (http://terrier.org/docs/v5.2/javadoc/org/terrier/matching/models/package-summary.html) pour voir s’il existent des méthodes que vous donnent de meilleurs résultats, c’est-à-dire inspectez les 10 premiers résultats et vérifiez si les pages d’intérêt (Thomas Müller, Mario Gómez ou Mario Götze et FC Bayern Munich) font partie des résultats, puis les 20 premiers, et finalement les 50 premiers. Existent-ils de différences importantes ?

**BM25:** 


In [25]:
pt.BatchRetrieve(indexrefResume, wmodel="BM25", metadata=["docno","title","url"]).transform("Thomas and Mario are strikers playing in Munich").head(50)

,qid,docid,docno,title,url,rank,score,query
0,1,54535,54536,Wikipedia: Paper Mario,https://simple.wikipedia.org/wiki/Paper_Mario,0,18.699940,Thomas and Mario are strikers playing in Munich
1,1,98207,98208,Wikipedia: Christian Bentez,https://simple.wikipedia.org/wiki/Christian_Be...,1,17.387904,Thomas and Mario are strikers playing in Munich
2,1,71326,71327,Wikipedia: Gernot Jurtin,https://simple.wikipedia.org/wiki/Gernot_Jurtin,2,17.387904,Thomas and Mario are strikers playing in Munich
3,1,131294,131295,Wikipedia: Munich RE,https://simple.wikipedia.org/wiki/Munich_RE,3,16.909863,Thomas and Mario are strikers playing in Munich
4,1,104224,104225,Wikipedia: Enner Valencia,https://simple.wikipedia.org/wiki/Enner_Valencia,4,16.829015,Thomas and Mario are strikers playing in Munich
5,1,52660,52661,Wikipedia: Gary Twigg,https://simple.wikipedia.org/wiki/Gary_Twigg,5,16.829015,Thomas and Mario are strikers playing in Munich
6,1,20273,20274,Wikipedia: Mario (disambiguation),https://simple.wikipedia.org/wiki/Mario_(disam...,6,16.779104,Thomas and Mario are strikers playing in Munich
7,1,44657,44658,Wikipedia: Flrin Albert,https://simple.wikipedia.org/wiki/Fl%C3%B3ri%C...,7,16.405210,Thomas and Mario are strikers playing in Munich
8,1,144124,144125,Wikipedia: Jean-Claude Schindelholz,https://simple.wikipedia.org/wiki/Jean-Claude_...,8,16.198575,Thomas and Mario are strikers playing in Munich
9,1,122975,122976,"Wikipedia: Bobby Murdoch (footballer, born 1936)",https://simple.wikipedia.org/wiki/Bobby_Murdoc...,9,16.198575,Thomas and Mario are strikers playing in Munich


**BM11:** 

In [26]:
pt.BatchRetrieve(indexrefResume, wmodel="BM25", metadata=["docno","title","url"]).transform("Thomas and Mario are strikers playing in Munich").head(50)

,qid,docid,docno,title,url,rank,score,query
0,1,54535,54536,Wikipedia: Paper Mario,https://simple.wikipedia.org/wiki/Paper_Mario,0,18.699940,Thomas and Mario are strikers playing in Munich
1,1,98207,98208,Wikipedia: Christian Bentez,https://simple.wikipedia.org/wiki/Christian_Be...,1,17.387904,Thomas and Mario are strikers playing in Munich
2,1,71326,71327,Wikipedia: Gernot Jurtin,https://simple.wikipedia.org/wiki/Gernot_Jurtin,2,17.387904,Thomas and Mario are strikers playing in Munich
3,1,131294,131295,Wikipedia: Munich RE,https://simple.wikipedia.org/wiki/Munich_RE,3,16.909863,Thomas and Mario are strikers playing in Munich
4,1,104224,104225,Wikipedia: Enner Valencia,https://simple.wikipedia.org/wiki/Enner_Valencia,4,16.829015,Thomas and Mario are strikers playing in Munich
5,1,52660,52661,Wikipedia: Gary Twigg,https://simple.wikipedia.org/wiki/Gary_Twigg,5,16.829015,Thomas and Mario are strikers playing in Munich
6,1,20273,20274,Wikipedia: Mario (disambiguation),https://simple.wikipedia.org/wiki/Mario_(disam...,6,16.779104,Thomas and Mario are strikers playing in Munich
7,1,44657,44658,Wikipedia: Flrin Albert,https://simple.wikipedia.org/wiki/Fl%C3%B3ri%C...,7,16.405210,Thomas and Mario are strikers playing in Munich
8,1,144124,144125,Wikipedia: Jean-Claude Schindelholz,https://simple.wikipedia.org/wiki/Jean-Claude_...,8,16.198575,Thomas and Mario are strikers playing in Munich
9,1,122975,122976,"Wikipedia: Bobby Murdoch (footballer, born 1936)",https://simple.wikipedia.org/wiki/Bobby_Murdoc...,9,16.198575,Thomas and Mario are strikers playing in Munich


**TF-IDF:**

In [27]:
pt.BatchRetrieve(indexrefResume, wmodel="TF_IDF", metadata=["docno","title","url"]).transform("Thomas and Mario are strikers playing in Munich").head(50)

,qid,docid,docno,title,url,rank,score,query
0,1,54535,54536,Wikipedia: Paper Mario,https://simple.wikipedia.org/wiki/Paper_Mario,0,10.232392,Thomas and Mario are strikers playing in Munich
1,1,98207,98208,Wikipedia: Christian Bentez,https://simple.wikipedia.org/wiki/Christian_Be...,1,9.496236,Thomas and Mario are strikers playing in Munich
2,1,71326,71327,Wikipedia: Gernot Jurtin,https://simple.wikipedia.org/wiki/Gernot_Jurtin,2,9.496236,Thomas and Mario are strikers playing in Munich
3,1,131294,131295,Wikipedia: Munich RE,https://simple.wikipedia.org/wiki/Munich_RE,3,9.231149,Thomas and Mario are strikers playing in Munich
4,1,104224,104225,Wikipedia: Enner Valencia,https://simple.wikipedia.org/wiki/Enner_Valencia,4,9.215769,Thomas and Mario are strikers playing in Munich
5,1,52660,52661,Wikipedia: Gary Twigg,https://simple.wikipedia.org/wiki/Gary_Twigg,5,9.215769,Thomas and Mario are strikers playing in Munich
6,1,20273,20274,Wikipedia: Mario (disambiguation),https://simple.wikipedia.org/wiki/Mario_(disam...,6,9.159570,Thomas and Mario are strikers playing in Munich
7,1,44657,44658,Wikipedia: Flrin Albert,https://simple.wikipedia.org/wiki/Fl%C3%B3ri%C...,7,8.959547,Thomas and Mario are strikers playing in Munich
8,1,119586,119587,Wikipedia: Giuseppe Virgili,https://simple.wikipedia.org/wiki/Giuseppe_Vir...,8,8.870533,Thomas and Mario are strikers playing in Munich
9,1,144124,144125,Wikipedia: Jean-Claude Schindelholz,https://simple.wikipedia.org/wiki/Jean-Claude_...,9,8.870533,Thomas and Mario are strikers playing in Munich


**TF-IDF/PL2:**

In [28]:
pt.BatchRetrieve(indexrefResume, wmodel="TF_IDF", metadata=["docno","title","url"], features=["WMODEL:TF_IDF", "WMODEL:PL2"]).transform("Thomas and Mario are strikers playing in Munich").head(50)

100%|██████████| 1/1 [00:00<00:00,  6.71q/s]


,qid,docid,docno,title,url,rank,score,query
0,1,54535,54536,Wikipedia: Paper Mario,https://simple.wikipedia.org/wiki/Paper_Mario,0,10.232392,Thomas and Mario are strikers playing in Munich
1,1,98207,98208,Wikipedia: Christian Bentez,https://simple.wikipedia.org/wiki/Christian_Be...,1,9.496236,Thomas and Mario are strikers playing in Munich
2,1,71326,71327,Wikipedia: Gernot Jurtin,https://simple.wikipedia.org/wiki/Gernot_Jurtin,2,9.496236,Thomas and Mario are strikers playing in Munich
3,1,131294,131295,Wikipedia: Munich RE,https://simple.wikipedia.org/wiki/Munich_RE,3,9.231149,Thomas and Mario are strikers playing in Munich
4,1,104224,104225,Wikipedia: Enner Valencia,https://simple.wikipedia.org/wiki/Enner_Valencia,4,9.215769,Thomas and Mario are strikers playing in Munich
5,1,52660,52661,Wikipedia: Gary Twigg,https://simple.wikipedia.org/wiki/Gary_Twigg,5,9.215769,Thomas and Mario are strikers playing in Munich
6,1,20273,20274,Wikipedia: Mario (disambiguation),https://simple.wikipedia.org/wiki/Mario_(disam...,6,9.159570,Thomas and Mario are strikers playing in Munich
7,1,44657,44658,Wikipedia: Flrin Albert,https://simple.wikipedia.org/wiki/Fl%C3%B3ri%C...,7,8.959547,Thomas and Mario are strikers playing in Munich
8,1,119586,119587,Wikipedia: Giuseppe Virgili,https://simple.wikipedia.org/wiki/Giuseppe_Vir...,8,8.870533,Thomas and Mario are strikers playing in Munich
9,1,144124,144125,Wikipedia: Jean-Claude Schindelholz,https://simple.wikipedia.org/wiki/Jean-Claude_...,9,8.870533,Thomas and Mario are strikers playing in Munich


**BB2:**

In [29]:
pt.BatchRetrieve(indexrefResume, wmodel="BB2", metadata=["docno","title","url"]).transform("Thomas and Mario are strikers playing in Munich").head(50)

,qid,docid,docno,title,url,rank,score,query
0,1,54535,54536,Wikipedia: Paper Mario,https://simple.wikipedia.org/wiki/Paper_Mario,0,10.900517,Thomas and Mario are strikers playing in Munich
1,1,20273,20274,Wikipedia: Mario (disambiguation),https://simple.wikipedia.org/wiki/Mario_(disam...,1,9.684069,Thomas and Mario are strikers playing in Munich
2,1,63635,63636,Wikipedia: Swedish Grand Prix,https://simple.wikipedia.org/wiki/Swedish_Gran...,2,9.300274,Thomas and Mario are strikers playing in Munich
3,1,74737,74738,Wikipedia: Club Deportivo Universidad Catlica,https://simple.wikipedia.org/wiki/Club_Deporti...,3,9.300274,Thomas and Mario are strikers playing in Munich
4,1,27097,27098,Wikipedia: Super Mario Land 2: 6 Golden Coins,https://simple.wikipedia.org/wiki/Super_Mario_...,4,9.300274,Thomas and Mario are strikers playing in Munich
5,1,53638,53639,Wikipedia: Sorrento Calcio,https://simple.wikipedia.org/wiki/Sorrento_Calcio,5,9.300274,Thomas and Mario are strikers playing in Munich
6,1,55195,55196,Wikipedia: Mario & Luigi: Superstar Saga,https://simple.wikipedia.org/wiki/Mario_%26_Lu...,6,9.164989,Thomas and Mario are strikers playing in Munich
7,1,131294,131295,Wikipedia: Munich RE,https://simple.wikipedia.org/wiki/Munich_RE,7,9.121149,Thomas and Mario are strikers playing in Munich
8,1,59807,59808,Wikipedia: Super Mario,https://simple.wikipedia.org/wiki/Super_Mario,8,9.042673,Thomas and Mario are strikers playing in Munich
9,1,104224,104225,Wikipedia: Enner Valencia,https://simple.wikipedia.org/wiki/Enner_Valencia,9,8.838514,Thomas and Mario are strikers playing in Munich


Peut importe le model, on ne trouve pas les bons résultats avec le résumé. Je pense que cela est du au fait que le résumé n'est pas forcément très précis. 

### 3) Utilisation de termes
Utilisez individuellement chaque terme de la requête pour voir si vous arrivez à trouver les pages d’intérêt. Réalisez la même vérification de résultats comme dans le point précédent.


**BM25:** 

In [31]:
query = "Thomas and Mario are strikers playing in Munich"
termToSearch = [x for x in query.split() if x[0]==x[0].upper()]
print(termToSearch)
for term, page in zip(termToSearch,['Thomas Muller','Mario Gtze', 'FC Bayern Munich']):
  dfres = pt.BatchRetrieve(indexrefResume, wmodel="BM25", metadata=["docno","title","url"]).transform(term)[['title','score']]
  print(dfres.head(50))
  print(dfres[dfres['title']=='Wikipedia: '+page])

['Thomas', 'Mario', 'Munich']
                                                title      score
0                             Wikipedia: Tom Spurgeon  13.536466
1                         Wikipedia: Gospel of Thomas  12.716006
2                            Wikipedia: M. T. Liggett  12.567112
3           Wikipedia: People's Party (United States)  12.567112
4                           Wikipedia: BSC Young Boys  12.567112
5                            Wikipedia: Thomas (name)  11.989321
6                            Wikipedia: Adrian Hasler  11.856870
7                         Wikipedia: Thomas Delahanty  11.856870
8                      Wikipedia: Governor of Vermont  11.856870
9                  Wikipedia: Frank-Walter Steinmeier  11.856870
10                           Wikipedia: Thomas Tallis  11.856870
11                              Wikipedia: Dan Rooney  11.856870
12                 Wikipedia: The Rescuers Down Under  11.856870
13                 Wikipedia: Thomas County, Nebraska  11.85

On remarque qu'avec la méthode des termes on obtient : 
* Mario Gtze en 99e position 
* FC Bayern Munich en 70e position 

En revanche on n'obtient pas Thomas Muller 

**TF-IDF:** 

In [38]:
query = "Thomas and Mario are strikers playing in Munich"
termToSearch = [x for x in query.split() if x[0]==x[0].upper()]
print(termToSearch)
for term, page in zip(termToSearch,['Thomas Muller','Mario Gtze', 'FC Bayern Munich']):
  dfres = pt.BatchRetrieve(indexrefResume, wmodel="TF_IDF", metadata=["docno","title","url"]).transform(term)[['title','score']]
  print(dfres.head(50))
  print(dfres[dfres['title']=='Wikipedia: '+page])

['Thomas', 'Mario', 'Munich']
                                                title     score
0                             Wikipedia: Tom Spurgeon  7.392062
1                         Wikipedia: Gospel of Thomas  6.944021
2                            Wikipedia: M. T. Liggett  6.862713
3           Wikipedia: People's Party (United States)  6.862713
4                           Wikipedia: BSC Young Boys  6.862713
5                            Wikipedia: Thomas (name)  6.547189
6                            Wikipedia: Adrian Hasler  6.474860
7                         Wikipedia: Thomas Delahanty  6.474860
8                      Wikipedia: Governor of Vermont  6.474860
9                  Wikipedia: Frank-Walter Steinmeier  6.474860
10                           Wikipedia: Thomas Tallis  6.474860
11                              Wikipedia: Dan Rooney  6.474860
12                 Wikipedia: The Rescuers Down Under  6.474860
13                 Wikipedia: Thomas County, Nebraska  6.474860
14        

On remarque qu'avec TF-IDF on obtient le même résultats que précédemment. La chose qui change c'est la valeur des scores mais cela ne change pas grandemment. 

**TF-IDF / BM25 :**

In [39]:
query = "Thomas and Mario are strikers playing in Munich"
termToSearch = [x for x in query.split() if x[0]==x[0].upper()]
print(termToSearch)
for term, page in zip(termToSearch,['Thomas Muller','Mario Gtze', 'FC Bayern Munich']):
  dfres = pt.BatchRetrieve(indexrefResume, metadata=["docno","title","url"], features = ["WMODEL : TF_IDF", "WMODEL : BM25"]).transform(term)[['title','score']]
  print(dfres.head(50))
  print(dfres[dfres['title']=='Wikipedia: '+page])

100%|██████████| 1/1 [00:00<00:00, 25.13q/s]

['Thomas', 'Mario', 'Munich']
                                                title     score
0                              Wikipedia: Thomas More  5.069931
1                            Wikipedia: Barbara Judge  5.069931
2                    Wikipedia: Inherent Vice (movie)  5.069077
3                       Wikipedia: Joyce Carol Thomas  5.067600
4                         Wikipedia: Space Master X-7  5.061021
5                         Wikipedia: Billy Mainwaring  5.034125
6                      Wikipedia: Thomas Haden Church  5.034125
7                     Wikipedia: Toby the Tram Engine  5.017404
8                             Wikipedia: Terry-Thomas  5.009530
9              Wikipedia: Mallappally Marthoma Church  4.998100
10                         Wikipedia: Oldfield Thomas  4.902468
11                                Wikipedia: Gorboduc  4.854428
12                 Wikipedia: Thomas Linley the elder  4.630683
13  Wikipedia: Mrs. Thomas Gainsborough, nee Marga...  4.373583
14        


100%|██████████| 1/1 [00:00<00:00, 99.10q/s]

                                             title     score
0                      Wikipedia: Super Mario Land  6.478612
1                       Wikipedia: Super Mario 128  6.197190
2       Wikipedia: Mario & Luigi: Partners in Time  6.197190
3                         Wikipedia: Mario Machado  6.194677
4                           Wikipedia: Hotel Mario  6.184448
5                        Wikipedia: Mario (series)  6.098275
6                          Wikipedia: Mario Kart 7  6.038799
7         Wikipedia: Mario & Luigi: Superstar Saga  5.973887
8                   Wikipedia: Super Mario Odyssey  5.953469
9             Wikipedia: New Super Mario Bros. Wii  5.897760
10                 Wikipedia: Mario Party (series)  5.830852
11                              Wikipedia: Waluigi  5.830852
12                                Wikipedia: Wario  5.830852
13                      Wikipedia: Boo (character)  5.830852
14                          Wikipedia: Paper Mario  5.652520
15                      

Ici on remarque que Mario Gtze n'a pas changer il est toujours à la 70e position, en revanche FC Bayern Munich se trouve en 11 e position. On remarque donc une amélioration lorsque l'on combine les deux modèles. 

### 4) Le titre
Maintenant utilisez le champ ***title*** avec la requête complète et avec chaque terme. Expliquez  pourquoi les résultats sont différents.

**Phrase entière BM25:** 

In [40]:
pt.BatchRetrieve(indexrefTitre, wmodel="BM25", metadata=["docno","title","url"]).transform("Thomas and Mario are strikers playing in Munich").head(50)

,qid,docid,docno,title,url,rank,score,query
0,1,28335,28336,Wikipedia: Mario Strikers Charged,https://simple.wikipedia.org/wiki/Mario_Strike...,0,22.935140,Thomas and Mario are strikers playing in Munich
1,1,28768,28769,Wikipedia: Striker,https://simple.wikipedia.org/wiki/Striker,1,17.920504,Thomas and Mario are strikers playing in Munich
2,1,578,579,Wikipedia: Munich,https://simple.wikipedia.org/wiki/Munich,2,15.932557,Thomas and Mario are strikers playing in Munich
3,1,131294,131295,Wikipedia: Munich RE,https://simple.wikipedia.org/wiki/Munich_RE,3,15.932557,Thomas and Mario are strikers playing in Munich
4,1,168074,168075,Wikipedia: Armsel Striker,https://simple.wikipedia.org/wiki/Armsel_Striker,4,15.554996,Thomas and Mario are strikers playing in Munich
5,1,34129,34130,Wikipedia: Matt Striker,https://simple.wikipedia.org/wiki/Matt_Striker,5,15.554996,Thomas and Mario are strikers playing in Munich
6,1,28416,28417,Wikipedia: Munich Agreement,https://simple.wikipedia.org/wiki/Munich_Agree...,6,13.829458,Thomas and Mario are strikers playing in Munich
7,1,100901,100902,Wikipedia: Munich massacre,https://simple.wikipedia.org/wiki/Munich_massacre,7,13.829458,Thomas and Mario are strikers playing in Munich
8,1,11134,11135,Wikipedia: Munich U-Bahn,https://simple.wikipedia.org/wiki/Munich_U-Bahn,8,13.829458,Thomas and Mario are strikers playing in Munich
9,1,11174,11175,Wikipedia: Munich Airport,https://simple.wikipedia.org/wiki/Munich_Airport,9,13.829458,Thomas and Mario are strikers playing in Munich


Sur la phrase entière on ne trouve pas de résultat pertinent avec le champ title. 

**Chaque terme BM25:** 

In [43]:
query = "Thomas and Mario are strikers playing in Munich"
termToSearch = [x for x in query.split() if x[0]==x[0].upper()]
print(termToSearch)
for term, page in zip(termToSearch,['Thomas Muller','Mario Gtze', 'FC Bayern Munich']):
  dfres = pt.BatchRetrieve(indexrefTitre, metadata=["docno","title","url"], features = ["WMODEL : BM25"]).transform(term)[['title','score']]
  print(dfres.head(50))
  print(dfres[dfres['title']=='Wikipedia: '+page])

100%|██████████| 1/1 [00:00<00:00, 31.87q/s]

['Thomas', 'Mario', 'Munich']
                                                title     score
0   Wikipedia: Thomas Pelham-Holles, 1st Duke of N...  3.394046
1   Wikipedia: Thomas Jefferson High School for Sc...  3.394046
2   Wikipedia: Thomas Seymour, 1st Baron Seymour o...  3.394046
3   Wikipedia: Mrs. Thomas Gainsborough, nee Marga...  3.394046
4       Wikipedia: Thomas Cromwell, 1st Earl of Essex  3.278270
5   Wikipedia: Thomas Jefferson National Accelerat...  3.278270
6      Wikipedia: Thomas Robinson, 1st Baronet Rokeby  3.278270
7        Wikipedia: Keith Thomas (English footballer)  3.096002
8                 Wikipedia: Thomas Edwin Blanton Jr.  3.096002
9              Wikipedia: Earl Thomas (wide receiver)  3.096002
10        Wikipedia: Saint Thomas Church (Strasbourg)  3.096002
11  Wikipedia: First inauguration of Thomas Jefferson  3.096002
12  Wikipedia: Second inauguration of Thomas Jeffe...  3.096002
13             Wikipedia: Saint-Thomas-de-Courceriers  3.096002
14        


100%|██████████| 1/1 [00:00<00:00, 134.01q/s]

                                                title     score
0       Wikipedia: Super Mario Land 2: 6 Golden Coins  3.859160
1   Wikipedia: Mario & Sonic at the Sochi 2014 Oly...  3.859160
2           Wikipedia: Wario Land: Super Mario Land 3  3.768554
3   Wikipedia: Mario and Sonic at the Olympic Game...  3.768554
4      Wikipedia: Super Mario World 2: Yoshi's Island  3.768554
5      Wikipedia: Paper Mario: The Thousand-Year Door  3.632262
6      Wikipedia: List of video games featuring Mario  3.632262
7           Wikipedia: New Super Mario Bros. U Deluxe  3.632262
8   Wikipedia: New Super Mario Bros. (disambiguation)  3.632262
9                  Wikipedia: New Super Mario Bros. 2  3.632262
10  Wikipedia: Super Mario RPG: Legend of the Seve...  3.632262
11      Wikipedia: Super Mario Bros.: The Lost Levels  3.632262
12               Wikipedia: New Super Mario Bros. Wii  3.632262
13  Wikipedia: Mario and Donkey Kong: Minis on the...  3.632262
14                       Wikipedia: Mari

On remarque qu'à la différence de chaque terme sur le champ resume, on obtient dirrectement un résultat favorable pour FC Bayern Munich qui se trouve en 2e position. En revanche Mario Gtze se trouve plus loin qu'avec le resumé, à la 123e position. 

**Chaque terme TF-IDF/BM25:** 

In [44]:
query = "Thomas and Mario are strikers playing in Munich"
termToSearch = [x for x in query.split() if x[0]==x[0].upper()]
print(termToSearch)
for term, page in zip(termToSearch,['Thomas Muller','Mario Gtze', 'FC Bayern Munich']):
  dfres = pt.BatchRetrieve(indexrefTitre, metadata=["docno","title","url"], features = ["WMODEL : TF_IDF", "WMODEL : BM25"]).transform(term)[['title','score']]
  print(dfres.head(50))
  print(dfres[dfres['title']=='Wikipedia: '+page])

100%|██████████| 1/1 [00:00<00:00, 44.55q/s]

['Thomas', 'Mario', 'Munich']
                                                title     score
0   Wikipedia: Thomas Pelham-Holles, 1st Duke of N...  3.394046
1   Wikipedia: Thomas Jefferson High School for Sc...  3.394046
2   Wikipedia: Thomas Seymour, 1st Baron Seymour o...  3.394046
3   Wikipedia: Mrs. Thomas Gainsborough, nee Marga...  3.394046
4       Wikipedia: Thomas Cromwell, 1st Earl of Essex  3.278270
5   Wikipedia: Thomas Jefferson National Accelerat...  3.278270
6      Wikipedia: Thomas Robinson, 1st Baronet Rokeby  3.278270
7        Wikipedia: Keith Thomas (English footballer)  3.096002
8                 Wikipedia: Thomas Edwin Blanton Jr.  3.096002
9              Wikipedia: Earl Thomas (wide receiver)  3.096002
10        Wikipedia: Saint Thomas Church (Strasbourg)  3.096002
11  Wikipedia: First inauguration of Thomas Jefferson  3.096002
12  Wikipedia: Second inauguration of Thomas Jeffe...  3.096002
13             Wikipedia: Saint-Thomas-de-Courceriers  3.096002
14        


100%|██████████| 1/1 [00:00<00:00, 209.54q/s]

                                                title     score
0       Wikipedia: Super Mario Land 2: 6 Golden Coins  3.859160
1   Wikipedia: Mario & Sonic at the Sochi 2014 Oly...  3.859160
2           Wikipedia: Wario Land: Super Mario Land 3  3.768554
3   Wikipedia: Mario and Sonic at the Olympic Game...  3.768554
4      Wikipedia: Super Mario World 2: Yoshi's Island  3.768554
5      Wikipedia: Paper Mario: The Thousand-Year Door  3.632262
6      Wikipedia: List of video games featuring Mario  3.632262
7           Wikipedia: New Super Mario Bros. U Deluxe  3.632262
8   Wikipedia: New Super Mario Bros. (disambiguation)  3.632262
9                  Wikipedia: New Super Mario Bros. 2  3.632262
10  Wikipedia: Super Mario RPG: Legend of the Seve...  3.632262
11      Wikipedia: Super Mario Bros.: The Lost Levels  3.632262
12               Wikipedia: New Super Mario Bros. Wii  3.632262
13  Wikipedia: Mario and Donkey Kong: Minis on the...  3.632262
14                       Wikipedia: Mari

Il n'y a pas de changement par rapport à précédemment. 

On remarque qu'on obtient toujours pas Thomas Muller. 

### 5) L'outil
Développez un outil/méthode/système capable de recevoir comme entrée un paragraphe de texte et comme sortie une liste ordonné de pages Wikipédia pour chaque unité lexical (token ou mot) dans le paragraphe.

####**Version avec un simple model**

C'est une version qui réalise une recherche avec la champ résumé sur chaque termes de la requête. 

In [48]:
def systeme(query, model):  
  L = [] 
  termToSearch = [x for x in query.split() if x[0]==x[0].upper()]
  print(termToSearch)

  for term in termToSearch:
      #print(pt.BatchRetrieve(indexref2, wmodel=model, metadata=["docno","title","url"]).transform(term)[['title','score']]) 
      L.append(pt.BatchRetrieve(indexrefResume, wmodel=model, metadata=["docno","title","url"]).transform(term)[['title','score']])
  return L 


In [69]:
L = systeme("Thomas and Mario are strikers playing in Munich", "BM25")


for e in L : 
  j = 0 
  i = 1
  for j in range(len(e)): 
    #print(e['title'][j])
    if (e['title'][j] == "Wikipedia: Mario Gtze" or e['title'][j] == "Wikipedia: FC Bayern Munich" or e['title'][j] == "Wikipedia: Thomas Mller"): 
      print( str(i) + " " + e['title'][j])
    i = i + 1 
  

['Thomas', 'Mario', 'Munich']
100 Wikipedia: Mario Gtze
71 Wikipedia: FC Bayern Munich


Avec ce système, on remarque que Mario Gtze se trouve à la 100e position et FC Bayern Munich se trouve à la 71e position. 

####**Version avec une simple intersection Titre/Resume**

C'est une version qui réalise une intersection avec les résultats trouvés avec le champ titre et avec les résultats du champ résumé. 

In [65]:
def systemeSimpleIntersection(query, wmodel, head):  
  L = [] 
  termToSearch = [x for x in query.split() if x[0]==x[0].upper()]
  print(termToSearch)

  listewmodel = [] 
  #if  (len(wmodel) >= 1) : 
  #  for s in wmodel : 
   #   listewmodel.append("WMODEL:" + s)

  print(listewmodel)
  for term in termToSearch:
    Liste = [] 
    P = pt.BatchRetrieve(indexrefResume, model = wmodel, metadata=["docno","title","url"]).transform(term)[['title','score']]
    R = pt.BatchRetrieve(indexrefTitre, model = wmodel, metadata=["docno","title","url"]).transform(term)[['title','score']] 

    #print(pt.BatchRetrieve(indexref2, wmodel=model, metadata=["docno","title","url"]).transform(term)[['title','score']]) 
    L.append(pd.merge(P, R, on = ['title']).head(head))
  return L 

In [70]:
L = systemeSimpleIntersection("Thomas and Mario are strikers playing in Munich", "BM25", 100)

for e in L : 
  j = 0 
  i = 1 
  for j in range(len(e)): 
    #print(e['title'][j])
    if (e['title'][j] == "Wikipedia: Mario Gtze" or e['title'][j] == "Wikipedia: FC Bayern Munich" or e['title'][j] == "Wikipedia: Thomas Mller"): 
      print( str(i) + " " + e['title'][j])
    i = i + 1 

100%|██████████| 1/1 [00:00<00:00, 20.18q/s]

['Thomas', 'Mario', 'Munich']
[]



100%|██████████| 1/1 [00:00<00:00, 153.69q/s]

49 Wikipedia: Mario Gtze
7 Wikipedia: FC Bayern Munich


Avec cette version on obtient "FC Bayern Munich" en 7e position et "Mario Gtze" en 49e position. Cette version est mieux que la précédente. 

####**Version avec différentes intersections**

Cette version du système réalise une intersection titre/résumé mais aussi une intersection entre terme principaux (avec majuscule) et terme secondaire (sans majuscule). Cette version supprime les mots non importants (conjonction de coordination, mots de liaison, verbes d'états, etc). Elle supprime ses mots de façon "barbare" mais on pourrait utiliser un fichier contenant tous ces mots ou autre. 

In [71]:
def motConnexion(mot): 
  if (mot == 'and'): 
    return False
  if (mot == 'or'): 
    return False 
  if (mot == 'in'): 
    return False 
  if (mot == 'are'): 
    return False 

  return True 

def systemeIntersection(query, wmodel, head):  
  L = [] 
  termToSearch = [x for x in query.split() if x[0]==x[0].upper()]
  print(termToSearch)

  termA = [x for x in query.split() if motConnexion(x) and x[0]!=x[0].upper()]
  print(termA)

  liste = []
  for t in termA : 
    R = pt.BatchRetrieve(indexrefResume, model = wmodel, metadata=["docno","title","url"]).transform(t)[['title','score']]
    liste.append(R) 

  for term in termToSearch:
    R = pt.BatchRetrieve(indexrefResume, model = wmodel, metadata=["docno","title","url"]).transform(term)[['title','score']]
    T = pt.BatchRetrieve(indexrefTitre, model = wmodel, metadata=["docno","title","url"]).transform(term)[['title','score']] 

    for e in liste : 
      RP = pd.merge(R, e, on = ['title']) 

      if (len(RP) > 1) : 
        R = pd.merge(R, e, on = ['title'])

    res = pd.merge(R, T, on = ['title'])
    #print(pt.BatchRetrieve(indexref2, wmodel=model, metadata=["docno","title","url"]).transform(term)[['title','score']]) 
    L.append(res.head(head))
  return L 

In [72]:
L = systemeIntersection("Thomas and Mario are strikers playing in Munich", "BM25", 80)

for e in L : 
  j = 0 
  i = 1 
  for j in range(len(e)): 
    #print(e['title'][j])
    if (e['title'][j] == "Wikipedia: Mario Gtze" or e['title'][j] == "Wikipedia: FC Bayern Munich" or e['title'][j] == "Wikipedia: Thomas Mller"): 
      print( str(i) + " " + e['title'][j])
    i = i + 1 

100%|██████████| 1/1 [00:00<00:00, 119.75q/s]

['Thomas', 'Mario', 'Munich']
['strikers', 'playing']



100%|██████████| 1/1 [00:00<00:00, 226.90q/s]

2 Wikipedia: Mario Gtze
7 Wikipedia: FC Bayern Munich


Avec cet intersection sur les mots avec majuscules et les mots minuscules on obtient "FC Bayern Munich" en 7e position et "Mario Gtze" en 2e position. Cette version est encore meilleure que la précédente. 
On revanche, on ne trouve toujours pas Thomas Muller. 


On évaluera toutes ses méthodes dans le TP5. Ici j'ai utilisé le model BM25 mais on aurait pu essayer avec TF-IDF ou même les deux ensembles. 



---



En plus à l’information du TP, vous pouvez utiliser les ressources/idées suivants pour améliorer les résultats du système à développer :
- La combinaison de plusieurs méthodes de pondération c’est possible en utilisant les indications sur https://github.com/terrier-org/pyterrier/blob/master/pipelines.md.
- Comme mentionné dans le TP1, il existe plusieurs type de champs. On a utilisé celui de la classe TextField. Renseignez vous sur les autres (StringField par exemple) et ses différences.
- Les distances d’édition sont largement utilisés par les moteurs de recherche. Par exemple si vous tapez « paris sain germain » sur Google il va vous suggérer la requête « paris saint germain » l’équipe de football. Vous pouvez utiliser les distances d’édition pour trouver des entités avec une orthographe différent que celle sur Wikipédia. Une de distance très performantes est la distance Levenshtein implémentée dans la classe LuceneLevenshteinDistance du paquet lucene.search.spell.
- Les moteurs de recherche peuvent être utilisés pour calculer la similarité entre les mots ou entre les entités. Par exemple, sur Google si vous tapez « orange » et « carrot » il y a plus de résultats que si vous tapez « purple » et « carrot ». Vous pouvez utiliser cette information pour décider si deux pages Wikipédia font partie d’un même sujet.
- Utilisez le fichier [anchor_text_en.ttl](http://downloads.dbpedia.org/2015-10/core-i18n/en/anchor_text_en.ttl.bz2) qui contient les anchor texts  de la Wikipédia en anglais. Le format du fichier est décrit [ici](https://iai-group.github.io/DBpedia-Entity/index_details.html). Transformez-le dans ce format au autre similaire pour faire l'indexation:

|             Frequency                 	|       Wikipedia Page        	| Anchor Text 	|
|------------------------------	|---------------	|-----------	|
|162| \<Saturn_Award_for_Best_Supporting_Actor> |"Best Supporting Actor"|
|14 |\<André_Golke>| "Golke"|
|13 |\<Charles_Pinckney_National_Historic_Site> |"Charles Pinckney National Historic Site"|
|25 |\<Thomas_Manfredini>| "Manfredini"|
|13 |<Marek_Zieńczuk> |"Marek Zieńczuk"|
|14 |\<Sigurd_Ulveseth>| "Sigurd Ulveseth"|
|104| <Pole_Position_(video_game)>| "Pole Position"|
|10| <Bang_(band)>| "Bang"|
|44| <George_Susce_(catcher)>| "George Susce"|
|11| \<Kyle_Rankin>| "Kyle Rankin"|
|…|…|…|

Donc, la premier colonne correspond au nombre de fois que la chaîne de caractères de la troisième colonne a été utilisé pour faire allusion à la page Wikipédia de la deuxième colonne. C’est-à-dire que "Best Supporting Actor" a été utilisé 162 fois dans Wikipédia pour faire allusion à la page Wikipédia https://en.wikipedia.org/wiki/Saturn_Award_for_Best_Supporting_Actor . Le fichier anchor_text_en.tll  contient les informations de la Wikipédia en anglais et pas de la Wikipédia simple, donc certain titres n’ont pas de correspondance.

> Si vous avez d’autres idées, partagez-les ! Mais ne partagez pas votre code!!!